In [1]:
import torch
import numpy as np
import scipy.io
import h5py
import sklearn.metrics
from torch_geometric.data import Data, DataLoader
import torch.nn as nn
from scipy.ndimage import gaussian_filter
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import reset, uniform
import random
import torch.nn.functional as F

In [2]:
from timeit import default_timer

In [3]:
TRAIN_PATH = 'data/piececonst_r241_N1024_smooth1.mat'

In [4]:
TEST_PATH = 'data/piececonst_r241_N1024_smooth2.mat'

In [5]:
t1 = default_timer()

In [6]:
class MatReader(object):
    def __init__(self, file_path, to_torch=True, to_cuda=False, to_float=True):
        super(MatReader, self).__init__()
        
        self.to_torch = to_torch
        self.to_cuda = to_cuda
        self.to_float = to_float
        
        self.file_path = file_path
        
        self.data = None
        self.old_mat = None
        self._load_file()
        
    def _load_file(self):
        try:
            self.data = scipy.io.loadmat(self.file_path)
            self.old_mat = True
        except:
            self.data = h5py.File(self.file_path)
            self.old_mat = False
            
    def load_file(self, file_path):
        self.file_path = file_path
        self._load_file()
        
        if not self.old_mat:
            x = x[()]
            x = np.transpose(x, axes=range(len(x.shape)- 1, -1, -1))
            
        if self.to_float:
            x = x.astype(np.float32)
            
        if self.to_torch:
            x = torch.from_numpy(x)
            
            if self.to_cuda:
                x = x.cuda()
                
    def read_field(self, field):
        x = self.data[field]
        
        if not self.old_mat:
            x = x[()]
            x = np.transpose(x, axes=range(len(x.shape) - 1, -1, -1))
            
        if self.to_float:
            x = x.astype(np.float32)
        
        if self.to_torch:
            x = torch.from_numpy(x)
            
            if self.to_cuda:
                x = x.cuda()
                
        return x

In [7]:
reader_train = MatReader(TRAIN_PATH)

In [8]:
r = 4
ntrain = 100
train_a = reader_train.read_field('coeff')[:ntrain,::r,::r].reshape(ntrain,-1)

In [9]:
train_a_smooth = reader_train.read_field('Kcoeff')[:ntrain,::r,::r].reshape(ntrain,-1)
train_a_gradx = reader_train.read_field('Kcoeff_x')[:ntrain,::r,::r].reshape(ntrain,-1)
train_a_grady = reader_train.read_field('Kcoeff_y')[:ntrain,::r,::r].reshape(ntrain,-1)
train_u = reader_train.read_field('sol')[:ntrain,::r,::r].reshape(ntrain,-1)
train_u64 = reader_train.read_field('sol')[:ntrain,::r,::r].reshape(ntrain,-1)

In [10]:
ntest = 40
reader_test = MatReader(TEST_PATH)
test_a = reader_test.read_field('coeff')[:ntest,::r,::r].reshape(ntest,-1)
test_a_smooth = reader_test.read_field('Kcoeff')[:ntest,::r,::r].reshape(ntest,-1)
test_a_gradx = reader_test.read_field('Kcoeff_x')[:ntest,::r,::r].reshape(ntest,-1)
test_a_grady = reader_test.read_field('Kcoeff_y')[:ntest,::r,::r].reshape(ntest,-1)
test_u = reader_test.read_field('sol')[:ntest,::r,::r].reshape(ntest,-1)

In [11]:
class GaussianNormalizer(object):
    def __init__(self, x, eps=0.00001):
        super(GaussianNormalizer, self).__init__()
        
        self.mean = torch.mean(x)
        self.std = torch.std(x)
        self.eps = eps
        
    def encode(self,x):
        x = (x - self.mean) / (self.std + self.eps)
        return x
    
    def cuda(self):
        self.mean = self.mean.cuda()
        self.std = self.std.cuda()
        
    def decode(self, x, sample_idx=None):
        x = (x *(self.std + self.eps)) + self.mean
        
    def cup(self):
        self.mean = self.mean.cpu()
        self.std = self.std.cpu()

In [12]:
a_normalizer_train = GaussianNormalizer(train_a)
a_normalizer_test = GaussianNormalizer(test_a)
train_a = a_normalizer_train.encode(train_a)
test_a = a_normalizer_test.encode(test_a)

In [13]:
as_normalizer_train = GaussianNormalizer(train_a_smooth)
as_normalizer_test = GaussianNormalizer(test_a_smooth)
train_a_smooth = as_normalizer_train.encode(train_a_smooth)
test_a_smooth = as_normalizer_test.encode(test_a_smooth)

In [14]:
agx_normalizer_train = GaussianNormalizer(train_a_gradx)
agx_normalizer_test = GaussianNormalizer(test_a_gradx)
train_a_gradx = agx_normalizer_train.encode(train_a_gradx)
test_a_gradx = agx_normalizer_test.encode(test_a_gradx)

In [15]:
agy_normalizer_train = GaussianNormalizer(train_a_grady)
agy_normalizer_test = GaussianNormalizer(test_a_grady)
train_a_grady = agy_normalizer_train.encode(train_a_grady)
test_a_grady = agy_normalizer_test.encode(test_a_grady)

In [16]:
u_normalizer_train = GaussianNormalizer(train_u)
u_normalizer_test = GaussianNormalizer(test_u)
train_u = u_normalizer_train.encode(train_u)
test_u = u_normalizer_test.encode(test_u)

In [17]:
class SquareMeshGenerator(object):
    def __init__(self, real_space, mesh_size):
        super(SquareMeshGenerator, self).__init__()
        
        self.d = len(real_space)
        self.s = mesh_size[0]
        
        assert len(mesh_size) == self.d
        
        if self.d == 1:
            self.n = mesh_size[0]
            self.grid = np.linspace(real_space[0][0], real_space[0][1], self.n).reshape((self.n,1))
        else:
            self.n = 1
            grids = []
            for j in range(self.d):
                grids.append(np.linspace(real_space[j][0], real_space[j][1], mesh_size[j]))
                self.n *= mesh_size[j]
            self.grid = np.vstack([xx.ravel() for xx in np.meshgrid(*grids)]).T
        
    def ball_connectivity(self, r):
        pwd = sklearn.metrics.pairwise_distances(self.grid)
        self.edge_index = np.vstack(np.where(pwd <= r))
        self.n_edges = self.edge_index.shape[1]
        
        return torch.tensor(self.edge_index, dtype=torch.long)
    
    def get_grid(self):
        return torch.tensor(self.grid, dtype=torch.float)
    
    def attributes(self, f=None, theta=None):
        if f is None:
            if  theta is None:
                edge_attr = self.grid[self.edge_index.T].reshape((self.n_edges,-1))
            else:
                edge_attr = np.zeros((self.n_edges, 3*self.d))
                edge_attr[:,0:2*self.d] = self.grid[self.edge_index.T].reshape((self.n_edges,-1))
                edge_attr[:,2*self.d] = theta[self.edge_index[0]]
                edge_attr[:,2*self.d+1] = theta[self.edge_index[1]]
                
        else:
            xy = self.grid[self.edge_indx.T].reshape((self.n_edges,-1))
            if theta is None:
                edge_attr = f(xy[:,0:self.d], xy[:,self.d:])
            else:
                edge_attr = f(xy[:,0:self.d], xy[:,self.d:], theta[self.edge_index[0]], theta[self.edge_index[1]])
                
        return torch.tensor(edge_attr, dtype=torch.float)

In [18]:
radius_train = 0.1
s = int(((241 - 1)/r) + 1)
meshgenerator_train = SquareMeshGenerator([[0,1],[0,1]],[s,s])
edge_index_train = meshgenerator_train.ball_connectivity(radius_train)
grid_train = meshgenerator_train.get_grid()
data_train = []
for j in range(ntrain):
    edge_attr_train = meshgenerator_train.attributes(theta=train_a[j,:])
    data_train.append(Data(x = torch.cat([grid_train, train_a[j,:].reshape(-1,1), train_a_smooth[j,:].reshape(-1,1), train_a_gradx[j,:].reshape(-1,1)], 
                                         dim=1),
                           y=train_u[j,:], coeff=train_a[j,:], 
                          edge_index=edge_index_train, edge_attr=edge_attr_train))
    
print('train grid', grid_train.shape, 'edge_index', edge_index_train.shape, 'edge_attr', edge_attr_train.shape)

train grid torch.Size([3721, 2]) edge_index torch.Size([2, 376471]) edge_attr torch.Size([376471, 6])


In [19]:
radius_test = 0.1
s = int(((241 - 1)/r) + 1)
meshgenerator_test = SquareMeshGenerator([[0,1],[0,1]],[s,s])
edge_index_test = meshgenerator_test.ball_connectivity(radius_test)
grid_test = meshgenerator_test.get_grid()
data_test = []
for j in range(ntest):
    edge_attr_test = meshgenerator_test.attributes(theta=test_a[j,:])
    data_test.append(Data(x = torch.cat([grid_test, test_a[j,:].reshape(-1,1), test_a_smooth[j,:].reshape(-1,1), test_a_gradx[j,:].reshape(-1,1)], 
                                         dim=1),
                           y=test_u[j,:], coeff=test_a[j,:], 
                          edge_index=edge_index_test, edge_attr=edge_attr_test))
    
print('test grid', grid_test.shape, 'edge_index', edge_index_test.shape, 'edge_attr', edge_attr_test.shape)

test grid torch.Size([3721, 2]) edge_index torch.Size([2, 376471]) edge_attr torch.Size([376471, 6])


In [20]:
batch_size = 1
train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(data_test, batch_size=batch_size, shuffle=True)

/home/wangyaohong/.conda/envs/pytorch1.6/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [21]:
t2 = default_timer()

In [22]:
print('preprocessing finished, time used:', t2-t1)

preprocessing finished, time used: 24.198634035885334


In [23]:
device = torch.device('cuda')

In [24]:
class DenseNet(torch.nn.Module):
    def __init__(self, layers, nonlinearity, out_nonlinearity=None, normalize=False):
        super(DenseNet, self).__init__()
        
        self.n_layers = len(layers) - 1
        
        assert self.n_layers >= 1
        
        self.layers = nn.ModuleList()
        
        for j in range(self.n_layers):
            self.layers.append(nn.Linear(layers[j], layers[j+1]))
            
            if j != self.n_layers -1:
                if normalize:
                    self.layers.append(nn.BatchNorm1d(layers[j+1]))
                    
                self.layers.append(nonlinearity())
            
        if out_nonlinearity is not None:
            self.layers.append(out_nonlinearity())
            
        def forward(self, x):
            for _, l in enumerate(self.layers):
                x = l(x)
                
            return x
            

In [25]:
class NNConv_old(MessagePassing):
    def __init__(self, in_channels, out_channels, nn, aggr='add', root_weight=True, bias=True, **kwargs):
        super(NNConv_old, self).__init__(aggr=aggr, **kwargs)
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.nn = nn
        self.aggr = aggr
        
        if root_weight:
            self.root = Parameter(torch.Tensor(in_channels, out_channels))
        else:
            self.register_parameter('root', None)
            
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
            
        self.reset_parameters()
    
    def reset_parameters(self):
        reset(self.nn)
        size = self.in_channels
        random.uniform(size, self.root)
        random.uniform(size, self.bias)
        
    def forward(self, x, edge_index, edge_attr):
        x = x.unsqueeze(-1) if x.dim() == 1 else x
        pseudo = edge_attr.unsqueeze(-1) if edge_attr.dim() == 1 else edge_attr
        return self.propagate(edge_index, x=x, pseudo=pseudo)

In [26]:
class KernelNN(torch.nn.Module):
    def __init__(self, width, ker_width, depth, ker_in, in_width=1, out_width=1):
        super(KernelNN, self).__init__()
        self.depth = depth
        
        self.fc1 = torch.nn.Linear(in_width, width)
        
        kernel = DenseNet([ker_in, ker_width, ker_width, width**2], torch.nn.ReLU)
        self.conv1 = NNConv_old(width, width, kernel, aggr='mean')
        
        self.fc2 = torch.nn.Linear(width,1)
        
    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.fc1(x)
        
        for k in range(self.depth):
            x = F.relu(self.conv1(x, edge_index, edge_attr))
            
        x = self.fc2(x)
        
        return x

In [27]:
width = 64
ker_width = 1024
depth = 6
edge_features = 5
node_features = 5
model = KernelNN(width,ker_width,depth,edge_features,node_features)

In [28]:
learning_rate=0.00001
scheduler_step=50
scheduler_gamma=0.8
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

In [29]:
class LpLoss(object):
    def __init__(self, d=2, p=2, size_average=True, reduction=True):
        super(LpLoss, self).__init__()
        
        assert d > 0 and p > 0
        
        self.d = d
        self.p = p
        self.reduction = reduction
        self.size_average = size_average

In [30]:
myloss = LpLoss(size_average=False)
u_normalizer_train.cuda()

In [31]:
model.train()

KernelNN(
  (fc1): Linear(in_features=5, out_features=64, bias=True)
  (conv1): NNConv_old(
    (nn): DenseNet(
      (layers): ModuleList(
        (0): Linear(in_features=5, out_features=1024, bias=True)
        (1): ReLU()
        (2): Linear(in_features=1024, out_features=1024, bias=True)
        (3): ReLU()
        (4): Linear(in_features=1024, out_features=4096, bias=True)
      )
    )
  )
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [32]:
epochs = 200
ttrain = np.zeros((epochs, ))
ttest = np.zeros((epochs, ))

In [33]:
k = 1
for ep in range(epochs):
    t1=default_timer()
    train_mse = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model.forward(batch)
        mse = F.mse_loss(out.view(-1,1), batch.y.view(-1,1))
        loss = torch.norm(out.view(-1) - batch.y.view(-1),1)
        loss.backward()
        optimizer.step()
        train_mse += mse.item()
    
    scheduler.step()
    t2 = default_timer()
    
    model.eval()
    
    ttrain[ep] = train_mse/(ntrain * k)
    
    print(ep, 'time:', t2-t1, 'train_mse:', train_mse/len(train_loader))

0 time: 570.9456669762731 train_mse: 0.88536949634552
1 time: 571.1392276976258 train_mse: 0.8798746824264526
2 time: 572.4460258558393 train_mse: 0.8745942848920822
3 time: 569.2366931512952 train_mse: 0.8697229677438736
4 time: 568.9453137014061 train_mse: 0.8653474169969558
5 time: 542.2387691037729 train_mse: 0.8611087030172349
6 time: 579.5861213244498 train_mse: 0.8572472190856933
7 time: 610.3146708263084 train_mse: 0.8534218949079514
8 time: 605.5875156987458 train_mse: 0.8501126009225846
9 time: 607.4865091769025 train_mse: 0.846898519396782
10 time: 610.7819434609264 train_mse: 0.8439859318733215
11 time: 608.0910995071754 train_mse: 0.8412635719776154
12 time: 611.2883183509111 train_mse: 0.8387409228086472
13 time: 615.5026474269107 train_mse: 0.8364101773500443
14 time: 607.4928281893954 train_mse: 0.8343162977695465
15 time: 606.2502419613302 train_mse: 0.8323686122894287
16 time: 604.6898616058752 train_mse: 0.8305724358558655
17 time: 547.7946564871818 train_mse: 0.8289

143 time: 490.54050638340414 train_mse: 0.8065707820653916
144 time: 488.4459029054269 train_mse: 0.8064889991283417
145 time: 486.68805147055537 train_mse: 0.8063404870033264
146 time: 484.46492191497236 train_mse: 0.8062719571590423
147 time: 489.4265095088631 train_mse: 0.8062134033441544
148 time: 486.5509097734466 train_mse: 0.8060274970531464
149 time: 486.5228855255991 train_mse: 0.8058979469537735
150 time: 488.11828797869384 train_mse: 0.8058676946163178
151 time: 487.34633979108185 train_mse: 0.8057205021381378
152 time: 491.8656376255676 train_mse: 0.8056969559192657
153 time: 488.67711600568146 train_mse: 0.8056224399805069
154 time: 488.7006934322417 train_mse: 0.8054249799251556
155 time: 489.27165769506246 train_mse: 0.8053572922945023
156 time: 484.94096324499696 train_mse: 0.8052546662092209
157 time: 485.34092209488153 train_mse: 0.8052261704206467
158 time: 487.0796016706154 train_mse: 0.8051013106107712
159 time: 489.4705542018637 train_mse: 0.8050187629461288
160 t

In [34]:
batch_size2 = 1
t1 = default_timer()
test_mse = 0
with torch.no_grad():
    for batch in test_loader:
        out = model.forward(batch)
        test_mse += F.mse_loss(out.view(batch_size2,-1), 
                          batch.y.view(batch_size2, -1))
ttest[ep] = test_mse / ntest
t2 = default_timer()
print('time:', t2-t1, 'train_mse:', train_mse/len(train_loader), 'test:', test_mse/ntest)

time: 77.23782046046108 train_mse: 0.8013147395849228 test: tensor(0.8467)


In [35]:
np.savetxt("path_train_err.txt", ttrain)
np.savetxt("path_test_err.txt", ttest)